In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model40net/modelnet40_hdf5_2048/train1_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/test0_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/test_files.txt
/kaggle/input/model40net/modelnet40_hdf5_2048/train3_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train3_id2name.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train_files.txt
/kaggle/input/model40net/modelnet40_hdf5_2048/test1_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train0_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train3.h5
/kaggle/input/model40net/modelnet40_hdf5_2048/test1.h5
/kaggle/input/model40net/modelnet40_hdf5_2048/train1_id2name.json
/kaggle/input/model40net/modelnet40_hdf5_2048/test0.h5
/kaggle/input/model40net/modelnet40_hdf5_2048/train2_id2name.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train2_id2file.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train0_id2name.json
/kaggle/input/model40net/modelnet40_hdf5_2048/train1.

In [2]:
pip install robust-laplacian numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import h5py
import os
from scipy.linalg import eigh
from scipy.spatial import distance
from robust_laplacian import point_cloud_laplacian
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from sklearn.neighbors import KDTree
from scipy.sparse.linalg import eigsh 

# Define constants
D = 3  # Dimensionality
M = 1536  # Simplified point cloud size
k_init = 256  # Initial points for FPS
k_add = 256  # Points added per iteration
k_opt = 4  # Points for optimizing GP hyperparameters

# Define the FPS function from paste.txt
def gp_pcs_simplify(point_cloud, num_points):
    """
    Simplify a point cloud using Farthest Point Sampling.
    """
    simplified_cloud = [point_cloud[np.random.choice(point_cloud.shape[0])]]
    
    for _ in range(num_points - 1):
        dists = np.linalg.norm(point_cloud - simplified_cloud[-1], axis=1)
        next_point_idx = np.argmax(dists)
        simplified_cloud.append(point_cloud[next_point_idx])
        point_cloud = np.delete(point_cloud, next_point_idx, axis=0)
    
    return np.array(simplified_cloud)

class GaussianProcess:
    def __init__(self, kernel, noise_var=1e-6):
        self.kernel = kernel
        self.sigma_y2 = noise_var**2

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.K = self.kernel(X, X) + self.sigma_y2 * np.eye(len(X))
        self.K += np.eye(len(self.K)) * 1e-6

    def predict(self, X_test):
        K_test = self.kernel(X_test, self.X_train)
        K_inv = np.linalg.inv(self.K)
        mean = np.dot(K_test, np.dot(K_inv, self.y_train))
    
        # Extract diagonal of covariance matrix
        var = self.kernel(X_test, X_test) - np.dot(K_test, np.dot(K_inv, K_test.T))
        var = np.maximum(np.diag(var), 1e-6)  # Take only diagonal and ensure positive values
    
        return mean, var
    def log_marginal_likelihood(self, theta):
        sigma2, kappa, nu = theta
        self.kernel.theta = theta
        K = self.kernel(self.X_train, self.X_train) + self.sigma_y2 * np.eye(len(self.X_train))
        
        # Enhanced numerical stability
        jitter = 1e-3  # Increased from 1e-4
        K += np.eye(K.shape[0]) * jitter
        
        try:
            # Use Cholesky decomposition instead of direct inverse
            L = np.linalg.cholesky(K)
            alpha = np.linalg.solve(L.T, np.linalg.solve(L, self.y_train))
            log_det = 2 * np.sum(np.log(np.diag(L)))
        except np.linalg.LinAlgError:
            # Return -inf for invalid parameters
            return -np.inf
        
        # Stable likelihood calculation
        log_likelihood = (
            -0.5 * np.dot(self.y_train.T, alpha)
            - 0.5 * log_det
            - len(self.y_train)/2 * np.log(2*np.pi)
        )
        
        # Prevent invalid values
        if not np.isfinite(log_likelihood):
            return -np.inf
            
        return log_likelihood.item()


class RiemannianKernel:
    def __init__(self, sigma_y=0.73, kappa=2.32, nu=-1.72, eigenvalues=None, eigenfunctions=None):
        self.sigma_y = sigma_y
        self.kappa = kappa
        self.nu = nu
        self.theta = (sigma_y, kappa, nu)
        self.Cv = 1.0  # Normalizing constant
        self.eigenvalues = eigenvalues
        self.eigenfunctions = eigenfunctions

    def __call__(self, X1, X2):
        if self.eigenvalues is None or self.eigenfunctions is None:
            raise ValueError("Eigenvalues and eigenfunctions must be provided.")
        
        sigma_y, kappa, nu = self.theta
        result = 0
        for n in range(len(self.eigenvalues)):
            result += ((2 * nu / kappa**2) + self.eigenvalues[n])**(-nu - 1.5) * self.eigenfunctions[n, 0] * self.eigenfunctions[n, 0]
        return (sigma_y**2 / self.Cv) * result

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

def estimate_eigenvalues_and_eigenfunctions(vertices, faces, num_eigenvalues=1):
    """
    Compute eigenvalues and eigenfunctions, ensuring k_neighbors <= available points.
    """
    num_points = len(vertices)

    # Dynamically set k_neighbors to avoid exceeding available points
    k_neighbors = min(16, num_points - 1)  # Ensure at least 1 neighbor

    # Compute the Laplacian matrix safely
    L, M = point_cloud_laplacian(vertices, n_neighbors=k_neighbors)

    L = csr_matrix(L)  # Convert to sparse matrix
    
    try:
        U, s, Vt = svds(L, k=min(num_eigenvalues, len(L.toarray()) - 1))  # Avoid exceeding matrix size
    except Exception as e:
        print(f"SVD computation failed: {e}")
        return None, None

    return s, U

def optimize_gp_hyperparameters(gp, X_opt, y_opt):
    from scipy.optimize import minimize
    
    def neg_log_likelihood(theta):
        return -gp.log_marginal_likelihood(theta)  # Ensure it's a scalar

    # Modified optimization configuration
    result = minimize(
        neg_log_likelihood,
        x0=[0.3, 1.8, 1.2],
        method="Nelder-Mead",
        options={
            "maxiter": 200,
            "disp": True,
            "adaptive": True,
            "xatol": 1e-3,  # Relaxed tolerance
            "fatol": 0.1    # Increased from 0.05
        }
    )
    return result.x
    
def regularize_cov_matrix(cov_matrix, regularization_strength=1e-4):
    return cov_matrix + regularization_strength * np.eye(cov_matrix.shape[0], dtype=np.float32)

import numpy as np
from sklearn.neighbors import KDTree

def compute_surface_variation(points):
    """
    Computes surface variation for each point in each sample efficiently.
    :param points: NumPy array of shape (n_samples, 3, 2048), containing point clouds.
    :return: NumPy array of shape (n_samples, 2048) containing surface variations.
    """
    print("Started computing Surface Variation (SV) efficiently")
    
    points_array = np.array(points)  # Shape (n_samples, 3, 2048)
    points_array = np.transpose(points_array, (0, 2, 1))  # Change to (n_samples, 2048, 3)
    
    k_neighbors = 35  # Reduce neighborhood size for efficiency
    num_samples, num_points, _ = points_array.shape  # (n_samples, 2048, 3)

    variations = np.zeros((num_samples, num_points))

    for i in range(num_samples):
        sample = points_array[i]  # Shape (2048, 3)
        kdtree = KDTree(sample)  # KDTree for current sample
        _, idxs = kdtree.query(sample, k=k_neighbors + 1)  # Shape (2048, k+1)
        neighbors = sample[idxs[:, 1:]]  # Exclude self, shape (2048, k, 3)

        # Compute centroids for all neighborhoods at once
        centroids = np.mean(neighbors, axis=1, keepdims=True)  # Shape (2048, 1, 3)
        centered_neighbors = neighbors - centroids  # Shape (2048, k, 3)

        # Compute covariance matrices efficiently using broadcasting (not PCA)
        cov_matrices = np.einsum('ijk,ijl->ikl', centered_neighbors, centered_neighbors) / (k_neighbors - 1)  # (2048, 3, 3)

        # Compute only the smallest eigenvalue for each point's covariance matrix
        eigenvalues = np.array([eigh(cov, eigvals_only=True) for cov in cov_matrices])  # Shape (2048, 3)
        min_eigenvalues = eigenvalues[:, 0]  # Smallest eigenvalue for each point

        # Compute surface variation
        variations[i, :] = min_eigenvalues / np.sum(eigenvalues, axis=1)  # Normalize by total variance

        if i % 100 == 0:  # Reduce print frequency
            print(f"Processed {i} / {num_samples} samples...")
    print("Vatiations:\n",variations[:5],"\nVariation length (no. of samples):",len(variations),"\n num of points in each sample:",len(variations[0]))
    print("Done computing surface variations")
    return variations  # Shape (n_samples, 2048)

def get_neighbors(point, points, k=32):
    # Convert points to a NumPy array for array indexing
    points_array = np.array(points)
    
    # For simplicity, use Euclidean distance to find neighbors
    dists = euclidean_distances([point], points_array)[0]
    idxs = np.argsort(dists)[:k+1]
    return points_array[idxs[1:]]

def compute_cov_matrix(points):
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid
    return np.dot(centered_points.T, centered_points)

def greedy_subset_of_data(points, surface_variations, M, k_init, k_add):
    """
    Implements greedy subset selection sample-wise, following the GP-PCS research paper.
    :param points: (n, 2048, 3) NumPy array of point clouds.
    :param surface_variations: (n, 2048) NumPy array of surface variations.
    :param M: Number of points to retain in the simplified set per sample.
    :param k_init: Initial number of points selected per sample using FPS.
    :param k_add: Points added per iteration.
    :return: (n, M, 3) NumPy array of simplified point clouds.
    """
    n, num_points, _ = points.shape  # (n, 2048, 3)

    print("Initializing active set sample-wise using FPS...")
    active_set = [gp_pcs_simplify(points[i], k_init).tolist() for i in range(n)]  # List of lists

    # Initialize as NumPy arrays instead of lists
    remainder_set = [points[i].copy() for i in range(n)]  # Keep as NumPy arrays
    remainder_variations = [surface_variations[i].copy() for i in range(n)]  # NumPy arrays

    # ... existing FPS initialization code ...

    # Modified mask application
    for i in range(n):
        kdtree = KDTree(points[i])
        active_indices = kdtree.query(np.array(active_set[i]), k=1)[1].flatten()

        # Create boolean mask for remainder points
        mask = np.ones(points[i].shape[0], dtype=bool)
        mask[active_indices] = False

        # Apply mask to NumPy arrays
        remainder_set[i] = points[i][mask]
        remainder_variations[i] = surface_variations[i][mask]

    print("Done loading remainder set.")
    print("Optimizing GP hyperparameters...")

    # Select P_opt for GP Optimization
    P_opt = np.array([gp_pcs_simplify(points[i], k_opt) for i in range(n)])  # (n, k_opt, 3)
    eigenvalues_list, eigenfunctions_list = [], []

    for i in range(n):
        vertices = P_opt[i]
        faces = np.random.randint(0, len(vertices), size=(len(vertices), 3))
        eigenvalues, eigenfunctions = estimate_eigenvalues_and_eigenfunctions(vertices, faces)
        eigenvalues_list.append(eigenvalues)
        eigenfunctions_list.append(eigenfunctions)

    print("Fitting a single GP model for all samples...")
    mean_eigenvalues = np.mean(np.array(eigenvalues_list), axis=0)
    mean_eigenfunctions = np.mean(np.array(eigenfunctions_list), axis=0)

    gp = GaussianProcess(RiemannianKernel(eigenvalues=mean_eigenvalues, eigenfunctions=mean_eigenfunctions))

    P_opt_flat = P_opt.reshape(-1, 3)
    y_opt_flat = surface_variations[:, :k_opt].reshape(-1)

    # gp.fit(P_opt_flat, y_opt_flat)

    # theta_opt = optimize_gp_hyperparameters(gp, P_opt_flat, y_opt_flat)
    # gp.kernel.theta = theta_opt
    print("Optimized GP hyperparameters:", gp.kernel.theta)
    print("Started adding points based on selection criteria...")
    # # **Greedy Selection Loop** (Optimized)
    gp.fit(P_opt_flat, y_opt_flat)

    # theta_opt = optimize_gp_hyperparameters(gp, P_opt_flat, y_opt_flat)  # <-- Commented out
    gp.kernel.theta = [0.73, 2.32, -1.72]  # <-- Use fixed hyperparameters
    print("Using fixed GP hyperparameters:", gp.kernel.theta)
    print("Started adding points based on selection criteria...")

    max_iterations = 14400000
    iteration = 0
    batch_size = 4  # Start with 4, adjust based on memory
    kdtrees = [KDTree(active_set[i]) for i in range(n)]
    completed = [False]*n
    remainder_masks = [np.ones(len(r), dtype=bool) for r in remainder_set]
    
    while sum(completed) < n and iteration < max_iterations:
        iteration += 1
        
        # Progress tracking
        if iteration % 20 == 0:
            active_sizes = [len(a) for a in active_set]
            remaining = [np.sum(m) for m in remainder_masks]
            # print(f"Iter {iteration}: Active {active_sizes} | Remaining {remaining}")
    
        # Process in batches
        for batch_start in range(0, n, batch_size):
            batch_samples = range(batch_start, min(batch_start+batch_size, n))
            
            # Batch Processing
            batch_points = []
            batch_variations = []
            sample_sizes = []
            
            # Collect data for current batch
            for i in batch_samples:
                if not completed[i]:
                    valid_indices = np.where(remainder_masks[i])[0]
                    batch_points.append(remainder_set[i][valid_indices])
                    batch_variations.append(remainder_variations[i][valid_indices])
                    sample_sizes.append(len(valid_indices))
                else:
                    sample_sizes.append(0)
            
            if sum(sample_sizes) == 0:
                continue
                
            # Concatenate and fit GP
            batch_points_concat = np.concatenate(batch_points)
            batch_variations_concat = np.concatenate(batch_variations)
            
            # Critical Fix: Fit GP on current active set
            active_points = np.concatenate([active_set[i] for i in batch_samples if not completed[i]])
            active_variations = np.concatenate([surface_variations[i][:len(active_set[i])] 
                                              for i in batch_samples if not completed[i]])
            gp.fit(active_points, active_variations)  # Update model with current active set
            
            # Get predictions
            batch_mean, batch_var = gp.predict(batch_points_concat)
            
            # Process individual samples
            ptr = 0
            for i_idx, i in enumerate(batch_samples):
                if completed[i] or sample_sizes[i_idx] == 0:
                    continue
                    
                # Slice predictions
                end = ptr + sample_sizes[i_idx]
                mean = batch_mean[ptr:end]
                var = batch_var[ptr:end]
                ptr = end
                
                # Get current valid indices
                valid_indices = np.where(remainder_masks[i])[0]
                
                # Size validation
                if len(mean) != len(valid_indices):
                    min_size = min(len(mean), len(valid_indices))
                    mean = mean[:min_size]
                    var = var[:min_size]
                    valid_indices = valid_indices[:min_size]
                
                # Get corresponding variations
                current_variations = remainder_variations[i][valid_indices]
                
                # Selection criterion
                selection = np.sqrt(var) + np.abs(mean - current_variations)
                valid_k = min(k_add, len(selection))
                
                if valid_k == 0:
                    continue
                    
                # Select top candidates
                selected = np.argpartition(selection, valid_k-1)[:valid_k]
                selected_indices = valid_indices[selected]
                
                # Update sets
                add_count = min(valid_k, M - len(active_set[i]))
                active_set[i] = np.vstack([active_set[i], remainder_set[i][selected_indices[:add_count]]])
                
                # Update mask
                remainder_masks[i][selected_indices[:add_count]] = False
                
                # Update KDTree if needed
                if add_count > 0:
                    kdtrees[i] = KDTree(active_set[i])
                
                # Check completion
                if len(active_set[i]) >= M:
                    completed[i] = True


    print("Simplification complete. ")


    # Convert back to a NumPy array before returning
    return np.array([np.array(a[:M]) for a in active_set])  # Ensure (n, M, 3)

class PointNetFeature(nn.Module):
    def __init__(self, global_feature=True, feature_transform=False):
        super(PointNetFeature, self).__init__()
        self.global_feature = global_feature
        self.feature_transform = feature_transform
        
        # Convolutional layers
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        
        # Batch normalization
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        
        # Optional feature transformation
        if self.feature_transform:
            self.fstn = FeatureTransformNet()

    def forward(self, x):
        n_pts = x.size()[2]
        
        # First convolution block
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Optional feature transformation
        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2, 1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2, 1)
        
        # Additional convolution blocks
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        
        # Global max pooling
        x = torch.max(x, 2, keepdim=True)[0]
        
        return x.view(-1, 1024)

class PointNetClassification(nn.Module):
    def __init__(self, num_classes=40, feature_transform=False):
        super(PointNetClassification, self).__init__()
        
        # Feature extraction
        self.feature_extraction = PointNetFeature(
            global_feature=True, 
            feature_transform=feature_transform
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
        # Batch normalization and dropout
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        # Extract global features
        x = self.feature_extraction(x)
        
        # Fully connected layers
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

class FeatureTransformNet(nn.Module):
    def __init__(self, k=64):
        super(FeatureTransformNet, self).__init__()
        
        # Convolutional layers
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        
        # Batch normalization
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        batch_size = x.size(0)
        
        # Convolutional blocks
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Global max pooling
        x = torch.max(x, 2, keepdim=True)[0]
        
        # Fully connected layers
        x = F.relu(self.bn4(self.fc1(x.squeeze(-1))))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        
        # Create identity matrix
        iden = torch.eye(x.size(-1) // x.size(0)).view(1, -1).repeat(batch_size, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(batch_size, -1, x.size(-1) // batch_size)
        
        return x

class ModelNet40Dataset(torch.utils.data.Dataset):
    def __init__(self, directory, train=True):
        """
        Load ModelNet40 dataset from HDF5 files
        Args:
            directory (str): Path to directory containing HDF5 files
            train (bool): Whether to load training or test data
        """
        self.points1 = []
        self.labels = []
        
        # Find all HDF5 files in the directory with "train" in the filename
        h5_files = []
        for f in os.listdir(directory):
            if f.endswith('.h5') and ('train' in f.lower()):
                h5_files.append(f)
        
        print(f"Found {len(h5_files)} training files: {h5_files}")
        
        for filename in h5_files:
            filepath = os.path.join(directory, filename)
            with h5py.File(filepath, 'r') as f:
                # Assume the first dataset is points, second is labels
                points = f['data'][:]
                labels = f['label'][:]
                
                self.points1.append(points)
                self.labels.append(labels)
        
        # Concatenate data
        self.points1 = np.concatenate(self.points1, axis=0)
        self.labels = np.concatenate(self.labels, axis=0)
        
        # Flatten labels
        self.labels = self.labels.squeeze()
        
        # Transpose points to match model input (C, N) format
        self.points1 = torch.from_numpy(self.points1.transpose(0, 2, 1)).float()
        self.labels = torch.from_numpy(self.labels).long()

        self.points1 = self.points1.numpy()  # Convert torch tensor to NumPy array
        # Reshape points from (9843, 3, 2048) to (9843, 2048, 3)
        reshaped_points = np.transpose(self.points1, (0, 2, 1))
        print("Reshaped points shape",reshaped_points.shape)
        
        # Compute surface variations
        surface_variations = compute_surface_variation(self.points1)
        
        # Simplify point cloud using the reshaped points
        self.points = greedy_subset_of_data(reshaped_points, surface_variations, M, k_init, k_add)

        print("Simplified Point Cloud Shape:", self.points.shape)
        print(f"Loaded {len(self.labels)} training samples")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.points[idx].astype(np.float32), self.labels[idx]


def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    for points, labels in train_loader:
        points, labels = points.to(device).float(), labels.to(device)  
        points = points.permute(0, 2, 1)  # Swap (batch, num_points, 3) -> (batch, 3, num_points)
        
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        total_loss += loss.item()

    return total_loss / len(train_loader), correct / total_samples


def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for points, labels in val_loader:
            points, labels = points.to(device).float(), labels.to(device)  
            points = points.permute(0, 2, 1)  #  Fix shape
            
            outputs = model(points)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            total_loss += loss.item()

    return total_loss / len(val_loader), correct / total_samples


def main():
    # Hyperparameters
    batch_size = 16
    learning_rate = 0.001
    num_epochs = 75
    num_classes = 40

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Dataset paths (adjust as needed)
    dataset_paths = [
        '/kaggle/input/model40net/modelnet40_hdf5_2048',
        '/content/modelnet40_hdf5_2048',
        '.'  # Current directory
    ]

    # Find a valid dataset path
    dataset_path = None
    for path in dataset_paths:
        if os.path.exists(path):
            dataset_path = path
            print(f"Using dataset path: {dataset_path}")
            break

    if not dataset_path:
        raise ValueError("Could not find ModelNet40 dataset")

    # Create dataset (only training data)
    train_dataset = ModelNet40Dataset(dataset_path, train=True)
    
    # Create data loader
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=0
    )

    # Initialize model
    model = PointNetClassification(num_classes=num_classes).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-5)

    # Training loop
    print("Starting training...")
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Step the learning rate scheduler
        scheduler.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')

    print("Training complete!")

    # Save the model
    torch.save(model.state_dict(), 'pointnet_modelnet40.pth')
    print("Model saved to pointnet_modelnet40.pth")

if __name__ == '__main__':
    main()


Using device: cuda
Using dataset path: /kaggle/input/model40net/modelnet40_hdf5_2048
Found 5 training files: ['train3.h5', 'train1.h5', 'train4.h5', 'train2.h5', 'train0.h5']
Reshaped points shape (9843, 2048, 3)
Started computing Surface Variation (SV) efficiently
Processed 0 / 9843 samples...
Processed 100 / 9843 samples...
Processed 200 / 9843 samples...
Processed 300 / 9843 samples...
Processed 400 / 9843 samples...
Processed 500 / 9843 samples...
Processed 600 / 9843 samples...
Processed 700 / 9843 samples...
Processed 800 / 9843 samples...
Processed 900 / 9843 samples...
Processed 1000 / 9843 samples...
Processed 1100 / 9843 samples...
Processed 1200 / 9843 samples...
Processed 1300 / 9843 samples...
Processed 1400 / 9843 samples...
Processed 1500 / 9843 samples...
Processed 1600 / 9843 samples...
Processed 1700 / 9843 samples...
Processed 1800 / 9843 samples...
Processed 1900 / 9843 samples...
Processed 2000 / 9843 samples...
Processed 2100 / 9843 samples...
Processed 2200 / 984

In [4]:
import torch
import h5py
import numpy as np
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to load test data from HDF5 with verification steps
def load_h5_data(h5_filename):
    with h5py.File(h5_filename, 'r') as f:
        data = f['data'][:]
        labels = f['label'][:]
        print(f"Loaded {len(labels)} samples from {h5_filename}")
        print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")
        # Print some sample labels to verify
        print(f"First 10 labels: {labels.squeeze()[:10]}")
    return data, labels

# Locate test files - make sure we're using different files than training
dataset_paths = [
    '/kaggle/input/model40net/modelnet40_hdf5_2048',
    '/content/modelnet40_hdf5_2048',
    '.'  # Current directory
]

# Find a valid dataset path
dataset_path = None
for path in dataset_paths:
    if os.path.exists(path):
        dataset_path = path
        print(f"Found dataset at: {dataset_path}")
        break

if not dataset_path:
    raise ValueError("Could not find ModelNet40 dataset")

# List all HDF5 files to ensure we're using separate test files
h5_files = [f for f in os.listdir(dataset_path) if f.endswith('.h5')]
print(f"Available H5 files: {h5_files}")

# Explicitly use test files (not train files)
test_files = [f for f in h5_files if f.startswith('test')]
if not test_files:
    raise ValueError("No test files found in the dataset directory")

print(f"Using test files: {test_files}")
test_h5_file = os.path.join(dataset_path, test_files[1])

# Load test data
X_test, y_test = load_h5_data(test_h5_file)
y_test = np.squeeze(y_test)  # Ensure labels are correctly shaped

# Verify input format before conversion
print(f"Input X shape before conversion: {X_test.shape}")
print(f"Input y shape before conversion: {y_test.shape}")

# Convert to PyTorch tensors WITHOUT transpose to see if that's causing issues
X_test_raw = torch.tensor(X_test, dtype=torch.float32)
print(f"Raw tensor shape (should be [N, num_points, 3]): {X_test_raw.shape}")

# Now apply transformation correctly based on model expectation
# PointNet expects [batch, channels, num_points]
X_test = X_test_raw.permute(0, 2, 1)
print(f"Transformed tensor shape (should be [N, 3, num_points]): {X_test.shape}")

y_test = torch.tensor(y_test, dtype=torch.long)

# Load trained model
num_classes = 40
model = PointNetClassification(num_classes=num_classes).to(device)
model.load_state_dict(torch.load("pointnet_modelnet40.pth"))
model.eval()  # Set model to evaluation mode

# Perform testing in a single loop with detailed logging
correct = 0
total = len(y_test)
class_correct = [0] * num_classes
class_total = [0] * num_classes

print("\nStarting evaluation...")
with torch.no_grad():
    for i in range(total):
        # Get a single test sample
        sample_X = X_test[i].unsqueeze(0).to(device)
        sample_y = y_test[i].to(device)
        
        # Forward pass
        output = model(sample_X)
        
        # Get prediction
        _, predicted = torch.max(output.data, 1)
        prediction = predicted[0]
        
        # Track correctness
        correct_prediction = (prediction == sample_y)
        if correct_prediction:
            correct += 1
            class_correct[sample_y] += 1
        class_total[sample_y] += 1
        
        # Print detailed info for first few samples
        if i < 10:
            print(f"Sample {i}: Label={sample_y.item()}, Prediction={prediction.item()}, Correct={correct_prediction.item()}")
        
        # Print progress
        if (i+1) % 100 == 0:
            print(f"Processed {i+1}/{total} samples. Current accuracy: {correct/(i+1):.4f}")

# Compute and print final accuracy
accuracy = correct / total
print(f"\nOverall Test Accuracy: {accuracy:.4f} ({correct}/{total})")

# Print per-class accuracy for the first few classes
print("\nPer-class accuracy:")
for i in range(min(10, num_classes)):
    if class_total[i] > 0:
        class_acc = class_correct[i] / class_total[i]
        print(f"Class {i}: {class_acc:.4f} ({class_correct[i]}/{class_total[i]})")

# Verify the model architecture
print("\nModel architecture:")
print(model)

Using device: cuda
Found dataset at: /kaggle/input/model40net/modelnet40_hdf5_2048
Available H5 files: ['train3.h5', 'test1.h5', 'test0.h5', 'train1.h5', 'train4.h5', 'train2.h5', 'train0.h5']
Using test files: ['test1.h5', 'test0.h5']
Loaded 2048 samples from /kaggle/input/model40net/modelnet40_hdf5_2048/test0.h5
Data shape: (2048, 2048, 3), Labels shape: (2048, 1)
First 10 labels: [17 17  7 24 22 26  7  2  9 21]
Input X shape before conversion: (2048, 2048, 3)
Input y shape before conversion: (2048,)
Raw tensor shape (should be [N, num_points, 3]): torch.Size([2048, 2048, 3])
Transformed tensor shape (should be [N, 3, num_points]): torch.Size([2048, 3, 2048])

Starting evaluation...
Sample 0: Label=17, Prediction=17, Correct=True
Sample 1: Label=17, Prediction=17, Correct=True
Sample 2: Label=7, Prediction=7, Correct=True
Sample 3: Label=24, Prediction=24, Correct=True
Sample 4: Label=22, Prediction=22, Correct=True
Sample 5: Label=26, Prediction=26, Correct=True
Sample 6: Label=7, P

/tmp/ipykernel_31/4003296764.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("pointnet_modelnet40.pth"))


Processed 100/2048 samples. Current accuracy: 0.9300
Processed 200/2048 samples. Current accuracy: 0.9150
Processed 300/2048 samples. Current accuracy: 0.9033
Processed 400/2048 samples. Current accuracy: 0.8950
Processed 500/2048 samples. Current accuracy: 0.8880
Processed 600/2048 samples. Current accuracy: 0.8917
Processed 700/2048 samples. Current accuracy: 0.8957
Processed 800/2048 samples. Current accuracy: 0.9050
Processed 900/2048 samples. Current accuracy: 0.9022
Processed 1000/2048 samples. Current accuracy: 0.9070
Processed 1100/2048 samples. Current accuracy: 0.9091
Processed 1200/2048 samples. Current accuracy: 0.9083
Processed 1300/2048 samples. Current accuracy: 0.9077
Processed 1400/2048 samples. Current accuracy: 0.9079
Processed 1500/2048 samples. Current accuracy: 0.9060
Processed 1600/2048 samples. Current accuracy: 0.9081
Processed 1700/2048 samples. Current accuracy: 0.9029
Processed 1800/2048 samples. Current accuracy: 0.9000
Processed 1900/2048 samples. Current 